In [1]:
import os
os.chdir('/home/nmduy/CA683/Credit Project')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score 
import pandas as pd
import numpy as np
import random
from sklearn import metrics
random.seed(1509)

## READ DATA AND REMOVE NA
Read CSV and only keep the full-info cells

In [2]:
data = pd.read_csv('data/imputed_train.csv')
data_remove_id = data.drop(['SK_ID_CURR'], axis=1) if 'SK_ID_CURR' in list(data.columns) else data.copy()
data_remove_id = data_remove_id.drop(['DAYS_EMPLOYED_ANOM'], axis=1) if 'DAYS_EMPLOYED_ANOM' in list(data_remove_id.columns) else data_remove_id.copy()
print(data_remove_id.shape)
data_remove_id = data_remove_id.dropna()
data_remove_id = data_remove_id.reset_index(drop=True)
print(data_remove_id.shape)

(307511, 110)
(307511, 110)


## MAKE IT BALANCE
Create subset and maybe make it balance between classes within classes

In [3]:
##### MAKE IT BALANCE #####
random.seed(1509)

numb_pos_samples = data_remove_id['TARGET'].sum()
numb_neg_samples = len(data_remove_id) - numb_pos_samples
portion_pos_train = 0.8
portion_pos_val = 0.1
numb_pos_train = int(numb_pos_samples * portion_pos_train)
numb_pos_val = int(numb_pos_samples * portion_pos_val)
numb_pos_test = numb_pos_samples - numb_pos_train - numb_pos_val

index_pos = data_remove_id.index[data_remove_id['TARGET'] == 1].tolist()
index_neg = [i for i in range(len(data_remove_id)) if i not in index_pos]

data_pos = data_remove_id.iloc[np.asarray(index_pos), :]
data_neg = data_remove_id.iloc[np.asarray(index_neg), :]

##### shuffle #####
data_pos = data_pos.sample(frac=1)
data_neg = data_neg.sample(frac=1)

data_pos_train = data_pos.iloc[0:numb_pos_train,:]
data_pos_val = data_pos.iloc[numb_pos_train:numb_pos_train + numb_pos_val, :]
data_pos_test = data_pos.iloc[numb_pos_train + numb_pos_val:, :]

data_neg_train = data_neg.iloc[0:int(numb_pos_train*11), :]
data_neg_val = data_neg.iloc[int(numb_pos_train*11):-numb_pos_test, :]
data_neg_test = data_neg.iloc[-numb_pos_test:, :]

data_train = pd.concat([data_pos_train, data_neg_train], ignore_index=True)
data_val = pd.concat([data_pos_val, data_neg_val], ignore_index=True)
data_not_test = pd.concat([data_train, data_val], ignore_index=True)
data_test = pd.concat([data_pos_test, data_neg_test], ignore_index=True)

print(f"Numb train/val/test samples: {len(data_train)} / {len(data_val)} / {len(data_test)}")

print(data_train.shape)

Numb train/val/test samples: 238320 / 64225 / 4966
(238320, 110)


## REMOVE FT CONTAIN ONLY 1 VALUES
Sometimes the binary features contains only 1 value due to the dropna functions

In [4]:
##### REMOVE 1-VALUE FT #####
remove_ft = []
for col in list(data_not_test.drop('TARGET', axis=1).columns):
    if data_not_test[col].sum() == 0 or data_not_test[col].sum() == len(data_not_test[col]):
        remove_ft += [col]

0

In [5]:
print(remove_ft)        
print(len(remove_ft))

data_train = data_train.drop(remove_ft, axis=1)
data_val = data_val.drop(remove_ft, axis=1)
data_test = data_test.drop(remove_ft, axis=1)
data_not_test = data_not_test.drop(remove_ft, axis=1)

[]
0


In [80]:
remove_ft

['FLAG_MOBIL', 'FLAG_DOCUMENT_2']

In [6]:
X_train = data_train.drop(['TARGET'], axis = 1)
X_test = data_test.drop(['TARGET'], axis = 1)
X_val = data_val.drop(['TARGET'], axis = 1)
y_train = data_train['TARGET']
y_test = data_test['TARGET']
y_val = data_val['TARGET']
X_not_test = data_not_test.drop(['TARGET'], axis = 1)
y_not_test = data_not_test['TARGET']
print(X_train.shape)

(238320, 109)


## TRAIN

In [16]:
rfc = RandomForestClassifier(class_weight='balanced')
rfc.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## EVALUATION

In [12]:
def calculate_metric(gtnp, pdnp):
    # input are numpy vector
    total_samples = len(gtnp)
    #print(f"Total sample: {total_samples}")
    total_correct = np.sum(gtnp == pdnp)
    accuracy = total_correct / total_samples
    gt_pos = np.where(gtnp == 1)[0]
    gt_neg = np.where(gtnp == 0)[0]
    TP = np.sum(pdnp[gt_pos])
    TN = np.sum(1 - pdnp[gt_neg])
    FP = np.sum(pdnp[gt_neg])
    FN = np.sum(1 - pdnp[gt_pos])
    precision = TP / (TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*precision*recall/(precision+recall)
    metrics = {}
    metrics['accuracy'] = str(accuracy)
    metrics['precision'] = str(precision)
    metrics['recall'] = str(recall)
    metrics['f1'] = str(f1)
    metrics['tp'] = str(int(TP))
    metrics['tn'] = str(int(TN))
    metrics['fp'] = str(int(FP))
    metrics['fn'] = str(int(FN))

    return metrics

In [17]:
##### TRAIN PREDICTION #####
train_predict = rfc.predict(X_train)
metric = calculate_metric(np.asarray(list(y_train)), train_predict)
metric['auc'] = roc_auc_score(y_train,train_predict)
metric

{'accuracy': '0.9999412554548506',
 'precision': '1.0',
 'recall': '0.9992950654582075',
 'f1': '0.9996474084521232',
 'tp': '19846',
 'tn': '218460',
 'fp': '0',
 'fn': '14',
 'auc': 0.9996475327291037}

In [18]:
##### VALIDATE PREDICTION #####
val_predict = rfc.predict(X_val)
metric = calculate_metric(np.asarray(list(y_val)), val_predict)
metric['auc'] = roc_auc_score(y_val,val_predict)
metric

{'accuracy': '0.96141689373297',
 'precision': '0.75',
 'recall': '0.0024174053182917004',
 'f1': '0.004819277108433735',
 'tp': '6',
 'tn': '61741',
 'fp': '2',
 'fn': '2476',
 'auc': 0.5011925064911592}

In [19]:
##### TEST PREDICTION #####
test_predict = rfc.predict(X_test)
metric = calculate_metric(np.asarray(list(y_test)), test_predict)
metric['auc'] = roc_auc_score(y_test,test_predict)
metric

{'accuracy': '0.5006041079339508',
 'precision': '1.0',
 'recall': '0.0012082158679017317',
 'f1': '0.0024135156878519713',
 'tp': '3',
 'tn': '2483',
 'fp': '0',
 'fn': '2480',
 'auc': 0.5006041079339508}

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [23]:
X = data_remove_id.drop(['TARGET'], axis = 1)
y = data_remove_id['TARGET']
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='f1')

In [25]:
print("=== All F1 Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean F1 Score ===")
print("Mean F1 Score - Random Forest: ", rfc_cv_score.mean())

=== All F1 Scores ===
[0.00356083 0.00356083 0.00710059 0.00829384 0.00237389 0.00356718
 0.00474215 0.00593824 0.00237954 0.00474496]


=== Mean F1 Score ===
Mean F1 Score - Random Forest:  0.004626204403960232
